In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('kafka2hdfs') \
    .getOrCreate()

23/09/07 14:48:53 WARN Utils: Your hostname, yujinhoui-MacBookPro.local resolves to a loopback address: 127.0.0.1, but we couldn't find any external IP address!
23/09/07 14:48:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/07 14:48:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/07 14:48:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.sql.types import StructType, StructField, StringType


In [3]:
data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df=spark.createDataFrame(data,columns)

df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)


In [5]:
df.write.mode('overwrite').parquet("hdfs://localhost:9000/pg_20230907.parquet")

In [15]:
from pyspark.sql.functions import expr, base64

df_encrypted = df\
    .withColumn('SSN_Encrypted', expr(f"aes_encrypt(SSN, '{super_secret_key}', 'ECB')"))\
    .withColumn('SSN_Encrypted', base64('SSN_Encrypted'))

In [16]:
display(df_encrypted)

DataFrame[FirstName: string, LastName: string, SSN: string, SSN_Encrypted: string]

In [17]:
df_encrypted.show()

+---------+--------+-----------+--------------------+
|FirstName|LastName|        SSN|       SSN_Encrypted|
+---------+--------+-----------+--------------------+
|    James|   Smith|111-22-3333|Pt9H19OJlCiGB5pT6...|
|  Michael|   Jones|222-33-4444|BTKw0FOfOyN3lKcbf...|
|    Maria|    Anne|333-44-5555|R8BmsMLw/na7kS4Jg...|
|    James|   Smith|111-22-3333|Pt9H19OJlCiGB5pT6...|
+---------+--------+-----------+--------------------+


In [19]:
df_encrypted.groupby('FirstName','LastName','SSN_Encrypted').count().show()

+---------+--------+--------------------+-----+
|FirstName|LastName|       SSN_Encrypted|count|
+---------+--------+--------------------+-----+
|    James|   Smith|Pt9H19OJlCiGB5pT6...|    2|
|  Michael|   Jones|BTKw0FOfOyN3lKcbf...|    1|
|    Maria|    Anne|R8BmsMLw/na7kS4Jg...|    1|
+---------+--------+--------------------+-----+


In [22]:
df_encrypted.createOrReplaceTempView("TestTable")

In [24]:
query = 'SELECT FirstName, LastName, count(SSN_Encrypted) AS OrderCount FROM TESTTABLE group by FirstName, LastName, SSN_Encrypted '
spark.sql(query).show()

+---------+--------+----------+
|FirstName|LastName|OrderCount|
+---------+--------+----------+
|    James|   Smith|         2|
|  Michael|   Jones|         1|
|    Maria|    Anne|         1|
+---------+--------+----------+
